In [3]:
# BOSTON CASAS

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score

# Cargar los datos
df = pd.read_excel(r'C:\Users\USUARIO\Dropbox\Mi PC (i3)\Desktop\Ejemplos\boston_house_prices.xlsx', sheet_name='boston_house_prices - ES')

# 1. Análisis Exploratorio
print("="*50)
print("Análisis Exploratorio")
print("="*50)
print(f"\nDimensión del dataset: {df.shape}")
print("\nPrimeras 5 filas:")
print(df.head())
print("\nResumen estadístico:")
print(df.describe())
print("\nInformación de tipos de datos y nulos:")
print(df.info())

# Visualización de distribuciones
plt.figure(figsize=(12, 8))
df.hist(bins=20, layout=(4, 4), figsize=(16, 12))
plt.tight_layout()
plt.savefig('distribuciones.png')
plt.close()

# Correlación con la variable objetivo
corr = df.corr()['MEDV'].sort_values(ascending=False)
print("\nCorrelación con MEDV:")
print(corr)

# Matriz de correlación
plt.figure(figsize=(12, 10))
sns.heatmap(df.corr(), annot=True, fmt='.2f', cmap='coolwarm')
plt.title('Matriz de Correlación')
plt.tight_layout()
plt.savefig('correlacion.png')
plt.close()

# 2. Preprocesamiento
# Separar características y objetivo
X = df.drop('MEDV', axis=1)
y = df['MEDV']

# Dividir datos
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Escalar características
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 3. Modelado y Evaluación
models = {
    'Regresión Lineal': LinearRegression(),
    'Ridge (L2)': Ridge(alpha=1.0),
    'Lasso (L1)': Lasso(alpha=0.1),
    'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42),
    'Gradient Boosting': GradientBoostingRegressor(n_estimators=100, random_state=42)
}

results = []
for name, model in models.items():
    # Entrenamiento
    model.fit(X_train_scaled, y_train)
    
    # Predicción
    y_pred = model.predict(X_test_scaled)
    
    # Evaluación
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)
    
    # Validación cruzada
    cv_scores = cross_val_score(model, X_train_scaled, y_train, cv=5, scoring='r2')
    cv_r2_mean = np.mean(cv_scores)
    
    results.append({
        'Modelo': name,
        'RMSE': rmse,
        'R²': r2,
        'R² CV': cv_r2_mean
    })
    
    # Visualización de predicciones
    plt.figure(figsize=(8, 6))
    sns.regplot(x=y_test, y=y_pred, line_kws={'color': 'red'})
    plt.title(f'Predicciones vs Real - {name}')
    plt.xlabel('Valores Reales (MEDV)')
    plt.ylabel('Predicciones (MEDV)')
    plt.savefig(f'predicciones_{name.replace(" ", "_")}.png')
    plt.close()

# Resultados comparativos
results_df = pd.DataFrame(results)
print("\n" + "="*50)
print("Comparación de Modelos")
print("="*50)
print(results_df)

# 4. Análisis de Importancia de Características (Mejor modelo)
best_model = GradientBoostingRegressor(n_estimators=100, random_state=42)
best_model.fit(X_train_scaled, y_train)

feature_importance = pd.DataFrame({
    'Característica': X.columns,
    'Importancia': best_model.feature_importances_
}).sort_values('Importancia', ascending=False)

plt.figure(figsize=(10, 6))
sns.barplot(x='Importancia', y='Característica', data=feature_importance)
plt.title('Importancia de Características (Gradient Boosting)')
plt.tight_layout()
plt.savefig('importancia_caracteristicas.png')
plt.close()

# 5. Análisis de Errores
y_pred = best_model.predict(X_test_scaled)
residuals = y_test - y_pred

plt.figure(figsize=(10, 6))
sns.scatterplot(x=y_pred, y=residuals)
plt.axhline(y=0, color='r', linestyle='--')
plt.title('Análisis de Residuales')
plt.xlabel('Predicciones')
plt.ylabel('Residuales')
plt.savefig('residuales.png')
plt.close()

# Resultados clave
print("\n" + "="*50)
print("Hallazgos Clave:")
print("="*50)
print(f"- Característica más importante: {feature_importance.iloc[0]['Característica']}")
print(f"- Mejor modelo: Gradient Boosting (R²: {results_df[results_df['Modelo']=='Gradient Boosting']['R²'].values[0]:.4f})")
print(f"- Error promedio (RMSE): {results_df[results_df['Modelo']=='Gradient Boosting']['RMSE'].values[0]:.2f}")
print("- Variables con mayor correlación con MEDV:")
print(corr.head(3))

Análisis Exploratorio

Dimensión del dataset: (506, 14)

Primeras 5 filas:
       CRIM   ZN  INDUS  CHAS    NOX     RM    AGE     DIS  RAD  TAX  PTRATIO  \
0  38.35180  0.0   18.1     0  0.693  5.453  100.0  1.4896   24  666     20.2   
1  67.92080  0.0   18.1     0  0.693  5.683  100.0  1.4254   24  666     20.2   
2  25.04610  0.0   18.1     0  0.693  5.987  100.0  1.5888   24  666     20.2   
3   9.91655  0.0   18.1     0  0.693  5.852   77.8  1.5004   24  666     20.2   
4  45.74610  0.0   18.1     0  0.693  4.519  100.0  1.6582   24  666     20.2   

        B  LSTAT  MEDV  
0  396.90  30.59   5.0  
1  384.97  22.98   5.0  
2  396.90  26.77   5.6  
3  338.16  29.97   6.3  
4   88.27  36.98   7.0  

Resumen estadístico:
             CRIM          ZN       INDUS        CHAS         NOX          RM  \
count  506.000000  506.000000  506.000000  506.000000  506.000000  506.000000   
mean     3.613524   11.363636   11.136779    0.069170    0.554695    6.284634   
std      8.601545   23.